In [310]:
import pandas as pd
import random
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [311]:
df = pd.read_csv('../Datasets/Titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [312]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [313]:
df.shape

(891, 12)

In [402]:
categorical = ['Survived', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
numeric = ['Pclass','Age','SibSp','Parch','Fare']

In [315]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [316]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [317]:
df['Embarked'] = df['Embarked'].fillna('S')

### Jako, że w kolumnie Cabin brakujące wartości stanowią znaczną większość, będziemy prowadzić nad nimi rozważania w formie surowej

In [318]:
df['Embarked'] = df['Embarked'].replace('NaN',pd.NA)

In [319]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [320]:
df_copy = df.copy()


### Generujemy losowo brakujące wartości

In [321]:
df_copy = df.copy()
df_random_rows = df.sample(frac=0.1)
random_indexes = df_random_rows.index.tolist()
len(random_indexes)

89

In [322]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [323]:
work_columns = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked']
for i in random_indexes:
    chosen_column = random.choice(work_columns)
    df_copy.at[i, chosen_column] = pd.NA
df_copy.isna().sum()

PassengerId      0
Survived         0
Pclass          14
Name             7
Sex              6
Age              5
SibSp            9
Parch            8
Ticket          22
Fare            11
Cabin          687
Embarked         7
dtype: int64

In [324]:
df_copy.isna().sum()

PassengerId      0
Survived         0
Pclass          14
Name             7
Sex              6
Age              5
SibSp            9
Parch            8
Ticket          22
Fare            11
Cabin          687
Embarked         7
dtype: int64

### Brak preprocessingu -> usuwanie brakujących wartości

In [325]:
df_1 = df_copy
df_1.isna().sum().sum()

776

In [326]:
df_1 = df_1.dropna()
df_1

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
21,22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0,B42,S


In [327]:
y = df_1['Survived']
y

1      1
3      1
6      0
11     1
21     1
      ..
871    1
872    0
879    1
887    1
889    1
Name: Survived, Length: 183, dtype: int64

In [328]:
df_1= df_1.drop(categorical, axis=1)
df_1 = df_1 = df_1.apply(pd.to_numeric)
X = df_1

In [329]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [330]:
clf = xgboost.XGBClassifier(n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.631578947368421

In [331]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.5789473684210527

In [332]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.5789473684210527

### Scenariusz 2: Wypełnienie brakujących wartości w kolumnach liczbowych wartością średnią

In [333]:
df_2 = df_copy
for col in numeric:
    df_2[col] = df_2[col].fillna(df_2[col].mean())

In [334]:
y = df_2['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [335]:
df_2 = df_2.drop(categorical,axis=1)
df_2 = df_2 = df_2.apply(pd.to_numeric)
X = df_2

In [336]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [337]:
clf = xgboost.XGBClassifier(n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7222222222222222

In [338]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.7333333333333333

In [339]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.7

### Scenariusz 3: Wypełnienie brakujących wartości w kolumnach liczbowych wartością minimalną

In [340]:
df_3 = df_copy
for col in numeric:
    df_3[col] = df_3[col].fillna(df_3[col].min())

In [341]:
y = df_3['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [342]:
df_3 = df_3.drop(categorical,axis=1)
df_3 = df_3 = df_3.apply(pd.to_numeric)
X = df_3

In [343]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [344]:
clf = xgboost.XGBClassifier(n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7222222222222222

In [345]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.7333333333333333

In [346]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.7

### Scenariusz 4: Wypełnienie brakujących wartości w kolumnach liczbowych wartością maksymalną

In [347]:
df_4 = df_copy
for col in numeric:
    df_4[col] = df_4[col].fillna(df_4[col].max())

In [348]:
y = df_4['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [349]:
df_4 = df_4.drop(categorical,axis=1)
df_4 = df_4 = df_4.apply(pd.to_numeric)
X = df_4

In [350]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
clf = xgboost.XGBClassifier(n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7222222222222222

In [351]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.7333333333333333

In [352]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.7

### Scenariusz 5: Zamiana kolumn kategorycznych na liczbowe poprzez wykorzystanie LabelEncoder

In [353]:
from sklearn.preprocessing import LabelEncoder

In [354]:
df_5 = df_copy
df_5 = df_5.dropna()
to_be_encoded = [
    "Name",
    "Sex",
    "Embarked",
    "Ticket",
    "Cabin"
]
encoder = LabelEncoder()

for col in to_be_encoded:
  df_5[col] = encoder.fit_transform(df_5[col])


df_5.describe()

C:\Users\Darek\AppData\Local\Temp\ipykernel_6140\3668231407.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5[col] = encoder.fit_transform(df_5[col])


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,457.025773,0.664948,1.202201,96.500000,0.520619,35.328245,0.461743,0.447199,67.139175,76.666088,71.231959,1.298969
std,250.638820,0.473230,0.523531,56.147128,0.500867,14.805550,0.633553,0.732264,39.001743,75.038261,40.007419,0.945838
min,2.000000,0.000000,1.000000,0.000000,0.000000,0.920000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,263.250000,0.000000,1.000000,48.250000,0.000000,25.250000,0.000000,0.000000,34.000000,28.959375,37.250000,0.000000
50%,459.500000,1.000000,1.000000,96.500000,1.000000,34.000000,0.000000,0.000000,68.500000,55.670850,69.500000,2.000000
75%,688.000000,1.000000,1.000000,144.750000,1.000000,46.000000,1.000000,1.000000,100.750000,90.000000,106.750000,2.000000
max,890.000000,1.000000,3.000000,193.000000,1.000000,80.000000,3.000000,4.000000,135.000000,512.329200,138.000000,2.000000


In [355]:
y = df_5['Survived']
y

1      1
3      1
6      0
11     1
21     1
      ..
871    1
872    0
879    1
887    1
889    1
Name: Survived, Length: 194, dtype: int64

In [356]:
df_5 = df_5.apply(pd.to_numeric)
X = df_5

In [357]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [358]:
clf = xgboost.XGBClassifier(n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [359]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

1.0

In [360]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.6

### Scenariusz 6: Zamiana kolumn kategorycznych na liczbowe poprzez wykorzystanie LabelEncoder, po wcześniejszym wypełnieniu braków najpopularniejszą wartością

In [361]:
df_6 = df_copy
to_be_encoded = [
    "Name",
    "Sex",
    "Embarked",
    "Ticket",
    "Cabin"
]


df_6.isna().sum().sum()


729

In [362]:
df_6 = df_6.fillna(df_6.mode().iloc[0])
df_6.isna().sum().sum()

0

In [363]:
encoder = LabelEncoder()

for col in to_be_encoded:
  df_6[col] = encoder.fit_transform(df_6[col])


In [372]:
y = df_6['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [373]:
df_6 = df_6.apply(pd.to_numeric)
X = df_6

In [374]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [375]:
clf = xgboost.XGBClassifier(n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [376]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

1.0

In [377]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.6555555555555556

### Scenariusz 7: Wypełnienie braków (liczbowe wartością średnią, kategoryczne najpopularniejszą), konwersja kolumn kategorycznych na liczbowe, modyfikacja kolumn w celu uzyskania większej ilości informacji

In [387]:
df_7 = df_copy
df_7.isna().sum().sum()

736

In [391]:
df_7['Title'] = df_7['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_7['Title'].value_counts()

Mr          514
Miss        181
Mrs         123
Master       39
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: Title, dtype: int64

In [393]:
df_7['Title'] = df_7['Title'].fillna(df_7['Title'].mode().iloc[0])
df_7['Title'].isna().sum()

0

In [395]:
df_7['Title'] = df_7['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_7['Title'] = df_7['Title'].replace('Mlle', 'Miss')
df_7['Title'] = df_7['Title'].replace('Ms', 'Miss')
df_7['Title'] = df_7['Title'].replace('Mme', 'Mrs')
df_7['Title'].value_counts()

Mr        521
Miss      184
Mrs       124
Master     39
Rare       23
Name: Title, dtype: int64

In [396]:
df_7 = df_7.drop(['Name','Ticket','PassengerId'],axis=1)
df_7

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3.0,male,22.000000,1.0,0.0,7.2500,NaN,S,Mr
1,1,1.0,female,38.000000,1.0,0.0,71.2833,C85,C,Mrs
2,1,3.0,female,26.000000,0.0,0.0,7.9250,NaN,S,Miss
3,1,1.0,female,35.000000,1.0,0.0,53.1000,C123,S,Mrs
4,0,3.0,male,35.000000,0.0,0.0,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...
886,0,2.0,male,27.000000,0.0,0.0,13.0000,NaN,S,Rare
887,1,1.0,female,19.000000,0.0,0.0,30.0000,B42,S,Miss
888,0,3.0,female,29.699118,1.0,2.0,23.4500,NaN,S,Miss
889,1,1.0,male,26.000000,0.0,0.0,30.0000,C148,C,Mr


In [399]:
df_7['Cabin'] = df_7['Cabin'].fillna('000')
df_7['Cabin'] = df_7['Cabin'].str[:1]
df_7['Cabin'].value_counts()

0    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: Cabin, dtype: int64

In [403]:
for col in numeric:
    df_7[col] = df_7[col].fillna(df_7[col].mean())

In [404]:
df_7 = df_7.fillna(df_7.mode().iloc[0])
df_7.isna().sum().sum()

0

In [406]:
to_be_encoded = [
    "Title",
    "Sex",
    "Embarked",
    "Cabin"
]

encoder = LabelEncoder()

for col in to_be_encoded:
  df_7[col] = encoder.fit_transform(df_7[col])

In [407]:
y = df_7['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [409]:
df_7 = df_7.apply(pd.to_numeric)
X = df_7

In [410]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [411]:
clf = xgboost.XGBClassifier(n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [412]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

1.0

In [413]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.8222222222222222